### Exercise 2: Plan-Execute + Reflection Hybrid

**Task:** Combine Plan-Execute and Reflection patterns to create a high-quality multi-step agent.

**Concept:** Use Plan-Execute to break down complex tasks, then use Reflection on the final output to ensure quality.

**Requirements:**
1. Create a hybrid state that includes:
   - Plan-Execute state (input, plan, current_step, results)
   - Reflection state (draft, critique, iterations)
2. Flow: Planner → Executor (loop) → Generator (uses results) → Critic → Refiner (if needed) → Finalizer
3. The Generator creates initial output from all executor results
4. The Critic evaluates the complete output (not individual steps)
5. Refine the final output based on critique (max 2 reflection iterations)
6. Track both execution progress AND reflection quality

**Architecture:**
```
START
  ↓
Planner (creates plan)
  ↓
Executor (executes each step)
  ↓ (loops until all steps done)
  ↓
Generator (synthesizes results into draft)
  ↓
Critic (evaluates quality)
  ↓
Should Refine?
  ↓              ↓
Refiner       Finalizer
  ↓              ↓
Critic         END
  (loops)
```

**Test scenario:**
```
Task: "Research the benefits of Python programming, create a summary, and make it beginner-friendly"

Expected flow:
1. PLAN: 
   - Step 1: Search for Python benefits
   - Step 2: Identify key benefits
   - Step 3: Create summary
2. EXECUTE: Complete all steps
3. GENERATE: Create initial summary from results
4. REFLECT: Critique summary for beginner-friendliness
5. REFINE: Simplify language and add examples
6. FINAL: High-quality, beginner-friendly summary
```

**Deliverables:**
1. Combined state definition (TypedDict)
2. All necessary nodes (planner, executor, generator, critic, refiner, finalizer)
3. Graph with proper edges and conditional routing
4. Test with the scenario above
5. Print output showing:
   - The plan
   - Each execution step
   - Initial draft
   - Critique
   - Refined version (if applicable)
   - Final output

---

In [2]:
# Install required packages
!pip install -q langgraph langchain langchain-openai python-dotenv

In [3]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

✅ All imports successful


In [4]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [5]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [6]:
from typing import TypedDict, List, Annotated
import json

# Define hybrid state combining Plan-Execute + Reflection
class HybridState(TypedDict):
    """Combined state for Plan-Execute + Reflection pattern"""
    # Input
    task: str
    
    # Plan-Execute components
    plan: List[str]  # List of planned steps
    current_step: int  # Current step being executed
    step_results: List[dict]  # Results from each step {step: str, result: str}
    
    # Reflection components
    draft: str  # Initial generated output
    critique: str  # Critic's evaluation
    refinement_count: int  # Track refinement iterations
    refined_output: str  # Refined version after critique
    final_output: str  # Final polished output


# Initialize LLM for this exercise
exercise_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print("✅ Hybrid State defined and LLM ready")

✅ Hybrid State defined and LLM ready


In [7]:
# Node 1: Planner - Creates a step-by-step plan
def planner(state: HybridState) -> dict:
    """Create a detailed plan for the task."""
    print("\n" + "="*60)
    print("🗂️  PLANNER NODE")
    print("="*60)
    
    prompt = f"""You are an expert planner. Analyze the following task and create a detailed step-by-step plan.

Task: {state['task']}

Create 3-4 specific, actionable steps to complete this task. Return ONLY a JSON array of steps, like:
["Step 1: Description", "Step 2: Description", "Step 3: Description"]"""
    
    response = exercise_llm.invoke([HumanMessage(content=prompt)])
    
    # Parse the plan
    try:
        # Extract JSON from response
        content = response.content.strip()
        if content.startswith("["):
            plan = json.loads(content)
        else:
            # Try to find JSON in the content
            start = content.find("[")
            end = content.rfind("]") + 1
            plan = json.loads(content[start:end])
    except:
        # Fallback plan
        plan = [
            "Step 1: Research the topic",
            "Step 2: Identify key points",
            "Step 3: Create summary",
            "Step 4: Review for clarity"
        ]
    
    print(f"\n📋 Plan created with {len(plan)} steps:")
    for i, step in enumerate(plan, 1):
        print(f"  {i}. {step}")
    
    return {
        "plan": plan,
        "current_step": 0,
        "step_results": []
    }


# Node 2: Executor - Executes each step in the plan
def executor(state: HybridState) -> dict:
    """Execute the current step of the plan."""
    if state["current_step"] >= len(state["plan"]):
        print("\n✅ All steps executed!")
        return {}
    
    step = state["plan"][state["current_step"]]
    print(f"\n{'='*60}")
    print(f"⚙️  EXECUTOR NODE - Step {state['current_step'] + 1}/{len(state['plan'])}")
    print(f"{'='*60}")
    print(f"📌 Executing: {step}")
    
    prompt = f"""You are an executor. Perform the following step in a detailed, thorough manner:

Step: {step}

Original Task: {state['task']}

Execute this step and provide a detailed result. Be specific and informative."""
    
    response = exercise_llm.invoke([HumanMessage(content=prompt)])
    result_text = response.content
    
    print(f"\n✓ Result:\n{result_text[:200]}..." if len(result_text) > 200 else f"\n✓ Result:\n{result_text}")
    
    # Store result
    new_results = state["step_results"].copy()
    new_results.append({
        "step": step,
        "result": result_text
    })
    
    return {
        "step_results": new_results,
        "current_step": state["current_step"] + 1
    }


# Node 3: Generator - Creates initial draft from all results
def generator(state: HybridState) -> dict:
    """Synthesize all step results into a coherent draft output."""
    print("\n" + "="*60)
    print("📝 GENERATOR NODE")
    print("="*60)
    
    # Prepare context from all steps
    results_text = "\n".join([
        f"Step {i+1}: {sr['step']}\nResult: {sr['result']}\n"
        for i, sr in enumerate(state["step_results"])
    ])
    
    prompt = f"""You are a content generator. Using the results from the execution steps below, create a coherent, well-structured draft output.

Task: {state['task']}

Execution Results:
{results_text}

Create a clear, comprehensive draft that synthesizes all these results into a single coherent output. The output should be well-organized and address the original task."""
    
    response = exercise_llm.invoke([HumanMessage(content=prompt)])
    draft = response.content
    
    print(f"\n📄 Draft generated ({len(draft)} characters)")
    print(f"\nDraft preview:\n{draft[:300]}..." if len(draft) > 300 else f"\nDraft:\n{draft}")
    
    return {
        "draft": draft,
        "refinement_count": 0
    }


# Node 4: Critic - Evaluates the draft output
def critic(state: HybridState) -> dict:
    """Evaluate the quality of the draft output."""
    print("\n" + "="*60)
    print("🔍 CRITIC NODE")
    print("="*60)
    
    prompt = f"""You are a critical evaluator. Review the following draft output against the original task requirement.

Task: {state['task']}

Draft Output:
{state['draft']}

Provide a critical evaluation including:
1. Strengths of the output
2. Areas for improvement
3. Specific suggestions for refinement
4. Overall quality assessment (rate as: Poor/Fair/Good/Excellent)

Focus especially on whether it's beginner-friendly and easy to understand."""
    
    response = exercise_llm.invoke([HumanMessage(content=prompt)])
    critique = response.content
    
    print(f"\n📊 Critique:")
    print(critique)
    
    return {
        "critique": critique
    }


# Node 5: Refiner - Refines output based on critique
def refiner(state: HybridState) -> dict:
    """Refine the draft based on critique."""
    print("\n" + "="*60)
    print("🔧 REFINER NODE - Iteration {state['refinement_count'] + 1}")
    print("="*60)
    
    prompt = f"""You are a refinement expert. Based on the critique below, improve and refine the draft output.

Original Task: {state['task']}

Current Draft:
{state['draft']}

Critic's Evaluation:
{state['critique']}

Create an improved version that addresses the critique. Focus on:
- Improving clarity and beginner-friendliness
- Adding concrete examples where helpful
- Simplifying complex language
- Making it more engaging and easier to understand"""
    
    response = exercise_llm.invoke([HumanMessage(content=prompt)])
    refined = response.content
    
    new_count = state["refinement_count"] + 1
    print(f"\n✨ Refined version (Iteration {new_count}):")
    print(refined[:300] + "..." if len(refined) > 300 else refined)
    
    return {
        "refined_output": refined,
        "refinement_count": new_count
    }


# Node 6: Finalizer - Completes the process
def finalizer(state: HybridState) -> dict:
    """Finalize the output."""
    print("\n" + "="*60)
    print("🎯 FINALIZER NODE")
    print("="*60)
    
    # Use refined output if available, otherwise use draft
    final = state.get("refined_output") or state["draft"]
    
    print("\n✅ Process complete!")
    print(f"Refinement iterations: {state['refinement_count']}")
    
    return {
        "final_output": final
    }

print("✅ All node functions defined")


✅ All node functions defined


In [8]:
# Conditional routing functions

def should_continue_executing(state: HybridState) -> Literal["executor", "generator"]:
    """Decide whether to continue executing or move to generation."""
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "generator"


def should_refine(state: HybridState) -> Literal["refiner", "finalizer"]:
    """Decide whether to refine based on critique quality."""
    # Check if critique suggests refinement is needed
    critique_lower = state["critique"].lower()
    
    # Signs that refinement is needed
    needs_refinement = any([
        "improve" in critique_lower,
        "could be better" in critique_lower,
        "simplify" in critique_lower,
        "more beginner-friendly" in critique_lower,
        "fair" in critique_lower,
        "good" in critique_lower
    ])
    
    # Don't refine more than 2 times
    if state["refinement_count"] >= 2:
        print("\n⏹️  Max refinement iterations reached (2)")
        return "finalizer"
    
    if needs_refinement:
        print("\n🔄 Refinement needed based on critique")
        return "refiner"
    
    print("\n✓ Output quality is excellent, proceeding to finalize")
    return "finalizer"


# Build the graph
builder = StateGraph(HybridState)

# Add nodes
builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("generator", generator)
builder.add_node("critic", critic)
builder.add_node("refiner", refiner)
builder.add_node("finalizer", finalizer)

# Add edges
builder.add_edge(START, "planner")
builder.add_conditional_edges("planner", should_continue_executing)
builder.add_conditional_edges("executor", should_continue_executing)
builder.add_edge("generator", "critic")
builder.add_conditional_edges("critic", should_refine)
builder.add_edge("refiner", "critic")  # Re-evaluate after refinement
builder.add_edge("finalizer", END)

# Compile the graph
hybrid_graph = builder.compile()

print("✅ Hybrid graph built successfully!")
print("\nGraph structure:")
print("START → Planner → Executor (loop) → Generator → Critic → (Refiner → Critic loop) → Finalizer → END")


✅ Hybrid graph built successfully!

Graph structure:
START → Planner → Executor (loop) → Generator → Critic → (Refiner → Critic loop) → Finalizer → END


In [9]:
# Test scenario: Research Python benefits with beginner-friendly output

test_task = "Research the benefits of Python programming, create a summary, and make it beginner-friendly"

print("🚀 Running Hybrid Plan-Execute + Reflection Agent")
print(f"📋 Task: {test_task}\n")

# Initial state
initial_state = {
    "task": test_task,
    "plan": [],
    "current_step": 0,
    "step_results": [],
    "draft": "",
    "critique": "",
    "refinement_count": 0,
    "refined_output": "",
    "final_output": ""
}

# Run the graph
final_state = hybrid_graph.invoke(initial_state)

print("\n\n" + "="*80)
print("📊 FINAL RESULTS SUMMARY")
print("="*80)

print("\n📋 PLAN:")
for i, step in enumerate(final_state["plan"], 1):
    print(f"  {i}. {step}")

print("\n\n⚙️  EXECUTION RESULTS:")
for i, result in enumerate(final_state["step_results"], 1):
    print(f"\n  Step {i}: {result['step']}")
    print(f"  Result: {result['result'][:150]}..." if len(result['result']) > 150 else f"  Result: {result['result']}")

print("\n\n📝 INITIAL DRAFT:")
print(f"\n{final_state['draft']}")

print("\n\n🔍 CRITIQUE:")
print(f"\n{final_state['critique']}")

if final_state["refinement_count"] > 0:
    print(f"\n\n✨ REFINED OUTPUT (Iterations: {final_state['refinement_count']}):")
    print(f"\n{final_state['refined_output']}")

print("\n\n🎯 FINAL OUTPUT:")
print(f"\n{final_state['final_output']}")

print("\n" + "="*80)
print(f"✅ Process completed successfully!")
print(f"   - Refinement iterations: {final_state['refinement_count']}")
print(f"   - Steps executed: {len(final_state['step_results'])}")
print("="*80)


🚀 Running Hybrid Plan-Execute + Reflection Agent
📋 Task: Research the benefits of Python programming, create a summary, and make it beginner-friendly


🗂️  PLANNER NODE

📋 Plan created with 4 steps:
  1. Step 1: Conduct a literature review by searching for reputable articles, blogs, and research papers on the benefits of Python programming. Focus on sources that highlight ease of learning, versatility, community support, and job market demand.
  2. Step 2: Compile the key benefits identified in the research into a structured format, categorizing them into sections such as 'Ease of Learning', 'Applications', 'Community Support', and 'Career Opportunities'.
  3. Step 3: Write a beginner-friendly summary by using simple language and relatable examples to explain each benefit. Aim for clarity and engagement to ensure that even those unfamiliar with programming can understand the advantages of learning Python.
  4. Step 4: Review the summary for clarity and conciseness, and seek feedback fr